In [12]:
from __future__ import division, print_function

In [13]:
!ls

cmd.sh					 labels.txt.gz
inputs.fa				 labels.txt.local.gz
label.intervals_file.tsv		 logs
label.intervals_file.tsv.gz		 prepare_data.ipynb
label_regions.ipynb			 prepare_data.py
label_regions.py			 prepare_data_tf.py
Labels coenrichment investigation.ipynb  splits
labels_pre_sorted.txt			 test_label
labels_pre.txt				 tflabel.intervals_file.tsv
labels_sorted.txt			 _tmp_IjCY3O
labels.txt


In [14]:
import gzip

f = gzip.open("labels.txt.gz", "rb")

labels = []
for i,line in enumerate(f):
    if (i > 0):
        line = line.rstrip()
        the_id, ctcf, six5, znf143 = line.split("\t")
        labels.append([int(ctcf), int(six5), int(znf143)])


In [15]:
import numpy as np
labels = np.array(labels)

In [16]:
print(len(labels))
print(np.sum(labels*(labels > -1),axis=0))

3535075
[358070  28870 297051]


In [17]:
ctcf_pos_mask = (labels[:,0]==1)
ctcf_neg_mask = (labels[:,0]==0)
six5_pos_mask = (labels[:,1]==1)
six5_neg_mask = (labels[:,1]==0)
znf143_pos_mask = (labels[:,2]==1)
znf143_neg_mask = (labels[:,2]==0)

#question 1: what is the coenrichment of CTCF with six5
#contingency table should look like:

# [ pos_for_ctcf_and_six5, pos_for_ctcf_but_not_six5]
# [ neg_for_ctcf_pos_for_six5, neg_for_ctcf_and_six5]

from scipy.stats import fisher_exact
print(fisher_exact(table=[
    [np.sum(ctcf_pos_mask*six5_pos_mask), np.sum(ctcf_pos_mask*six5_neg_mask)],
    [np.sum(ctcf_neg_mask*six5_pos_mask), np.sum(ctcf_neg_mask*six5_neg_mask)]
]))

#question 2: What is the coenrichment of CTCF with six5 in the presence of znf143
from scipy.stats import fisher_exact
print(fisher_exact(table=[
    [np.sum(ctcf_pos_mask*six5_pos_mask*znf143_pos_mask), np.sum(ctcf_pos_mask*six5_neg_mask*znf143_pos_mask)],
    [np.sum(ctcf_neg_mask*six5_pos_mask*znf143_pos_mask), np.sum(ctcf_neg_mask*six5_neg_mask*znf143_pos_mask)]
]))


(3.3277658403378623, 0.0)
(0.09825732060230231, 0.0)


In [18]:
(np.sum(ctcf_pos_mask*six5_pos_mask)/np.sum(ctcf_pos_mask*six5_neg_mask))/(
    np.sum(ctcf_neg_mask*six5_pos_mask)/np.sum(ctcf_neg_mask*six5_neg_mask))

3.3277658403378623